In [ ]:
import pandas as pd
# Load the full dataset (replace with your actual full data file path)
full_data_path = "C:/Users/win10/Desktop/US_Accidents_March23.csv"
df = pd.read_csv(full_data_path)

# Sample 1 million rows (or all if dataset smaller)
sample_size = min(1_000_000, len(df))
df_sampled = df.sample(n=sample_size, random_state=42)  # random_state for reproducibility

# Optional: reset index
df_sampled.reset_index(drop=True, inplace=True)

# Save sampled data
output_path = "US_Accidents_March23_sampled_1M.csv"
df_sampled.to_csv(output_path, index=False)

# print(f"Sampled {sample_size} rows saved to {output_path}")


In [ ]:
df = pd.read_csv('US_Accidents_March23_sampled_1M.csv')
df.head()

In [ ]:
df.info()

In [ ]:
(df.isnull().sum() / df.shape[0])*100

In [ ]:
# Drop the columns of 'End_Lat', 'End_Lng'
df = df.drop(columns=['End_Lat', 'End_Lng'])

In [ ]:
# Convert the 'Start_Time' and 'End_Time' columns to datetime format
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='mixed')
df['End_Time'] = pd.to_datetime(df['End_Time'], format='mixed')
df.info()

In [ ]:
# Convert coordinates to numeric (if not already)
df['Start_Lat'] = pd.to_numeric(df['Start_Lat'], errors='coerce')
df['Start_Lng'] = pd.to_numeric(df['Start_Lng'], errors='coerce')

In [ ]:
(df.isnull().sum() / df.shape[0])*100

In [ ]:
# Drop the rows with missing values except some columns.
df = df.dropna(subset=['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'Distance(mi)', 'Description', 'Street', 'City', 'County',
       'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code',
       'Weather_Timestamp', 'Temperature(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Direction','Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'])

In [ ]:
(df.isnull().sum() / df.shape[0])*100

In [ ]:
df[['Wind_Chill(F)', 'Wind_Speed(mph)', 'Precipitation(in)']].sample(5)

In [ ]:
(df[['Wind_Chill(F)', 'Wind_Speed(mph)', 'Precipitation(in)']].isnull().sum())/df.shape[0]*100

In [ ]:
# Median imputation for Wind_Speed(mph)
wind_median = df['Wind_Speed(mph)'].median()
df['Wind_Speed(mph)'] = df['Wind_Speed(mph)'].fillna(wind_median)

# Primary imputation for Precipitation(in): zero-fill
df['Precipitation(in)'] = df['Precipitation(in)'].fillna(0.0)

# Secondary imputation for known rain days: fill remaining gaps with median of nonzero precipitation
median_nonzero_precip = df.loc[df['Precipitation(in)'] > 0, 'Precipitation(in)'].median()

# Example mask for known rain days (replace with your actual condition)
# e.g., df['Rain_Flag'] == 1 or based on another indicator column
rain_day_mask = df['Precipitation(in)'].isna()  # placeholder if original missing flags retained

# Apply secondary imputation
df.loc[rain_day_mask, 'Precipitation(in)'] = median_nonzero_precip


In [ ]:
from sklearn.linear_model import LinearRegression

# Select features to predict Wind_Chill(F)
reg_features = ['Wind_Speed(mph)', 'Temperature(F)', 'Humidity(%)']  # adjust to available predictors

# Split known and unknown
known_wc = df[df['Wind_Chill(F)'].notna()]
unknown_wc = df[df['Wind_Chill(F)'].isna()]

# Train regression model
X_train = known_wc[reg_features]
y_train = known_wc['Wind_Chill(F)']
reg = LinearRegression()
reg.fit(X_train, y_train)

# Predict missing Wind_Chill(F)
X_pred = unknown_wc[reg_features]
predicted_wc = reg.predict(X_pred)

# Impute missing values
df.loc[df['Wind_Chill(F)'].isna(), 'Wind_Chill(F)'] = predicted_wc


In [ ]:
(df[['Wind_Chill(F)', 'Wind_Speed(mph)', 'Precipitation(in)']].isnull().sum())/df.shape[0]*100

In [ ]:
# 1. Parse datetimes
df["Start_Time"] = pd.to_datetime(df["Start_Time"], errors="coerce")
df["End_Time"]   = pd.to_datetime(df["End_Time"], errors="coerce")

In [ ]:
# 2. Drop duplicates and rows with invalid times
df = df.drop_duplicates(subset="ID")
df = df.dropna(subset=["Start_Time", "End_Time"])

In [ ]:
# 3. Drop rows missing critical location data
df = df.dropna(subset=["Start_Lat", "Start_Lng"])

In [ ]:
# 4. Compute incident duration in minutes
df["Duration_Minutes"] = (df["End_Time"] - df["Start_Time"]).dt.total_seconds() / 60

In [ ]:
# 5. Extract temporal features
df["Hour"]        = df["Start_Time"].dt.hour
df["DayOfWeek"]   = df["Start_Time"].dt.weekday
df["Month"]       = df["Start_Time"].dt.month
df["IsWeekend"]   = df["DayOfWeek"].isin([5,6]).astype(int)

In [ ]:
# 6. Encode boolean traffic feature flags as integers
bool_cols = [
    "Roundabout",
    "Station",
    "Stop",
    "Traffic_Calming",
    "Traffic_Signal",
    "Turning_Loop"
]
for col in bool_cols:
    df[col] = df[col].astype(int)

In [ ]:
# 7. Encode light condition as binary day/night
df["IsDay"] = (df["Sunrise_Sunset"] == "Day").astype(int)

In [ ]:
# 8. Drop columns not used for modeling
drop_cols = [
    "ID",
    "Source",
    "Description",
    "Street",
    "Start_Time",
    "End_Time",
    "Sunrise_Sunset",
    "Civil_Twilight",
    "Nautical_Twilight",
    "Astronomical_Twilight"
]
df = df.drop(columns=drop_cols)

In [ ]:
# 9. Handle missing values in numeric columns
#    Fill numeric NaNs with median
num_cols = df.select_dtypes(include="number").columns.tolist()
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

In [ ]:
# 10. Final clean-up: remove any remaining rows with NaNs
df = df.dropna()

In [ ]:
# 11. stratify/drop rare severity classes if needed
# e.g., keep only severity levels 1-4
df = df[df["Severity"].isin([1,2,3,4])]

# Save cleaned dataset
df.to_csv("accidents_cleaned.csv", index=False)